In [1]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1 # 2aB
nH = 10
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="crenbl", verbose=4)
mol.build()

mf = scf.RHF(mol)
e = mf.kernel()

mycc = cc.CCSD(mf)
e = mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Tue Sep 23 19:16:13 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 10
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.

In [ ]:
# def thouless_trans(t1):
#     # U, s, V = np.linalg.svd(t1.T)
#     # S = np.zeros_like(t1.T, dtype=float)
#     # np.fill_diagonal(S, s)
#     # u_ai = U@S
#     # u_ji = V.T
#     Q, R = np.linalg.qr(t1)
#     u_ai = R.T
#     u_ji = Q
#     u = np.vstack((u_ji,u_ai))
#     return u

In [ ]:
# Q, R = np.linalg.qr(t1)

In [ ]:
# print(Q.T)
# print(V)

[[-1.23012711e-13 -1.00000000e+00 -2.13201397e-13]
 [ 2.97775155e-01 -2.40134010e-13  9.54636034e-01]
 [-9.54636034e-01  5.39276097e-14  2.97775155e-01]]
[[-1.50352915e-01  5.12045188e-13 -9.88632389e-01]
 [ 2.86704375e-13  1.00000000e+00  4.74176254e-13]
 [ 9.88632389e-01 -2.12142207e-13 -1.50352915e-01]]


In [ ]:
# S = np.zeros_like(t1.T, dtype=float)
# np.fill_diagonal(S, s)
# print(np.max(U@S@V-t1.T))

3.570398377063447e-18


In [ ]:
# S = np.zeros_like(t1.T, dtype=float)
# np.fill_diagonal(S, s)
# u_ai = U@S
# u_ji = V.T
# # u_ai = R.T
# # u_ji = Q
# u = np.vstack((u_ji,u_ai))
# print(u.shape)

(24, 3)


In [ ]:
# s1e = mf.get_ovlp()
# c_psi = mf.mo_coeff
# c_phi = c_psi @ u
# print(np.linalg.det(c_phi.T@s1e@c_phi))

1.001013736034442


In [ ]:
# def mk_rdm1(mo_occ,s1e):
#     olp = np.linalg.det(mo_occ.T@s1e@mo_occ)
#     dm = 2*mo_occ.dot(mo_occ.conj().T)/olp
#     return dm

In [ ]:
# dm = mk_rdm1(c_phi,s1e)
# h1e = mf.get_hcore(mol)
# vhf = mf.get_veff(mol, dm)
# e_tot = mf.energy_tot(dm, h1e, vhf)
# print(e_tot)

-3.227208398613664


In [ ]:
# t2_0 = np.zeros(t2.shape)
# eris = mycc.ao2mo(mycc.mo_coeff)
# eccsd = mycc.energy(t1, t2_0, eris)
# print(eccsd+mf.e_tot)

-3.2313225574082045


In [4]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'rhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [3]:
from ad_afqmc.lno_ccsd import lno_maker

In [56]:
u = lno_maker.thouless_trans(mycc.t1)
# mo_t = mf.mo_coeff @ u
# mf.mo_coeff = mo_t

from ad_afqmc import pyscf_interface, config, mpi_jax, driver
import numpy as np

config.setup_jax()
MPI = config.setup_comm()

pyscf_interface.prep_afqmc(mf,chol_cut=1e-7)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

# init_walkers = np.stack([wave_data["mo_coeff"]] * nwalkers, axis=0)
# wave_data["mo_coeff"] = wave_data["mo_coeff"] @ u
# driver.afqmc(ham_data, ham, prop, trial, wave_data, sampler, observable, options, MPI,init_walkers)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (5, 5)
# Number of basis functions: 40
# Number of Cholesky vectors: 159
#
# Number of MPI ranks: 1
#
# norb: 40
# nelec: (5, 5)
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 2
# walker_type: rhf
# trial: rhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [57]:
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]


trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)
print(prop_data['e_estimate']-mf.e_tot)

2.1880630640680465e-09


In [58]:
# def thouless_trans(t1):
#     # U, s, V = np.linalg.svd(t1.T)
#     # S = np.zeros_like(t1.T, dtype=float)
#     # np.fill_diagonal(S, s)
#     # u_ai = U@S
#     # u_ji = V.T
Q, R = np.linalg.qr(mycc.t1)
u_ai = R.T
u_ji = Q
u_occ = np.vstack((u_ji,u_ai))

In [59]:
nocc = np.count_nonzero(mf.mo_occ)
wave_data['mo_coeff'] = u_occ

In [14]:
from ad_afqmc import wavefunctions

In [61]:
prop_data['walkers'][0]

Array([[-2.16177889e-31+0.j,  9.13269966e-32+0.j,  1.00000000e+00+0.j,
        -2.95660884e-32+0.j, -4.24607911e-32+0.j],
       [-4.65616612e-01+0.j, -4.03751807e-01+0.j,  3.66889460e-30+0.j,
         7.85581180e-01+0.j, -5.52074169e-02+0.j],
       [ 2.25335003e-01+0.j,  6.53248046e-03+0.j,  1.11942105e-30+0.j,
         2.03865419e-01+0.j,  9.52691111e-01+0.j],
       [ 6.04675332e-01+0.j,  4.71744784e-01+0.j, -1.04135934e-30+0.j,
         5.81819965e-01+0.j, -2.70758433e-01+0.j],
       [ 6.05634419e-01+0.j, -7.83835865e-01+0.j, -1.28189897e-30+0.j,
        -5.27885267e-02+0.j, -1.26576688e-01+0.j],
       [-7.79691020e-17+0.j, -2.39355314e-16+0.j,  2.57233390e-16+0.j,
        -7.74418315e-17+0.j, -3.17270160e-17+0.j],
       [ 2.43512652e-17+0.j,  3.20454175e-16+0.j, -1.27326387e-17+0.j,
         1.88948642e-16+0.j, -8.33532187e-17+0.j],
       [-3.95959626e-16+0.j, -2.58298722e-16+0.j,  1.04855725e-16+0.j,
         3.58840974e-16+0.j, -9.70706779e-17+0.j],
       [-2.05544501e-16+

In [72]:
e_rhf \
= wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                    )._calc_energy_restricted(wave_data['mo_coeff'], ham_data, wave_data)
print(e_rhf-mf.e_tot)

2.1880630640680465e-09


In [42]:
e_rhf \
= wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                    )._calc_energy_restricted(u_occ, ham_data, wave_data)
print(e_rhf-mf.e_tot)

-1.4911063734857066e-05


In [62]:
u = lno_maker.thouless_trans(mycc.t1)
s1e = mf.get_ovlp()
# c_phi = mf.mo_coeff @ u
# q, _ = np.linalg.qr(c_phi.T @ s1e @ c_phi)
q, _ = np.linalg.qr(mf.mo_coeff.T @ s1e @ mf.mo_coeff)

In [69]:
wave_data['mo_coeff'] = q[:,:nocc]
e_rhf \
= wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                    )._calc_energy_restricted(u_occ, ham_data, wave_data)
print(e_rhf-mf.e_tot)

-1.4911063734857066e-05


In [77]:
walker = wave_data['mo_coeff']
wave_data['mo_coeff'] = q[:,:nocc]
# walker = u[:,:nocc]
e_rhf \
= wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                    )._calc_energy_restricted(walker, ham_data, wave_data)
print(e_rhf-mf.e_tot)

2.1880630640680465e-09


In [82]:
from jax import vmap
from jax import numpy as jnp
# def calc_e_rhf(trial, walker, ham_data, wave_data):
walker = wave_data['mo_coeff']
wave_data['mo_coeff'] = q[:,:nocc]
h0, rot_h1, rot_chol = ham_data["h0"], ham_data["rot_h1"], ham_data["rot_chol"]
ene0 = h0
green_walker = trial._calc_green(walker, wave_data)
ene1 = 2.0 * jnp.sum(green_walker * rot_h1)
f = jnp.einsum("gij,jk->gik", rot_chol, green_walker.T, optimize="optimal")
c = vmap(jnp.trace)(f)
exc = jnp.sum(vmap(lambda x: x * x.T)(f))
ene2 = 2.0 * jnp.sum(c * c) - exc
# return ene2 + ene1 + ene0
print(ene0,ene1,ene2)
print(ene2 + ene1 + ene0 - mf.e_tot)

10.20766040588143 -25.54903593591203 9.970934122386547
2.1880630640680465e-09


In [ ]:
# from ad_afqmc.lno.cc import LNOCCSD
# import sys
# from ad_afqmc.lno_ccsd import lno_ccsd
# from pyscf.lib import logger
# from pyscf import lib
# log = logger.Logger(sys.stdout, 6)

In [ ]:
# mfcc = mycc
# thresh = 0
# frozen = None
# run_frg_list = [0]
# use_df_vecs = False
# chol_cut = 1e-8
# lo_type = 'boys'
# no_type = 'ie'
# frag_lolist = '1o'

# from pyscf.cc.ccsd import CCSD
# from pyscf.ci.cisd import CISD
# if isinstance(mfcc, (CCSD, CISD)):
#     full_cisd = True
#     mf = mfcc._scf
# else:
#     full_cisd = False
#     mf = mfcc

# if isinstance(thresh, list):
#     thresh_occ, thresh_vir = thresh
# else:
#     thresh_occ = thresh*10
#     thresh_vir = thresh

# lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen)
# lno_cc.thresh_occ = thresh_occ
# lno_cc.thresh_vir = thresh_vir
# lno_cc.lo_type = lo_type
# lno_cc.no_type = no_type
# lno_cc.frag_lolist = frag_lolist
# lno_cc.force_outcore_ao2mo = True

# s1e = lno_cc._scf.get_ovlp()
# orbactocc = lno_cc.split_mo()[1] # non-localized active occ
# orbloc = lno_cc.get_lo(lo_type=lo_type) # localized active occ orbitals
# m = orbloc.T @ s1e @ orbactocc
# lospanerr = abs((m.T @ m) - np.eye(m.shape[1])).max()
# if lospanerr > 1e-10:
#     log.error('LOs do not fully span the occupied space! '
#                 'Max|<occ|LO><LO|occ>| = %e', lospanerr)
#     raise RuntimeError
# # check 2: Span(LO) == Span(occ)
# occspanerr = abs(m @ m.T - np.eye(m.shape[0])).max()
# if occspanerr < 1e-10:
#     log.info('LOs span exactly the occupied space.')
#     if no_type not in ['ir','ie']:
#         log.error('"no_type" must be "ir" or "ie".')
#         raise ValueError
# else:
#     log.info('LOs span occupied space plus some virtual space.')

# if frag_lolist == '1o':
#     log.info('Using single-LO fragment') # this is what we use, every active local occ labels a fragment
#     frag_lolist = [[i] for i in range(orbloc.shape[1])]
# else: print('Only support single LO fragment!')
# nfrag = len(frag_lolist)

# if not (no_type[0] in 'rei' and no_type[1] in 'rei'):
#     log.warn('Input no_type "%s" is invalid.', no_type)
#     raise ValueError

# frozen_mask = lno_cc.get_frozen_mask()
# thresh_pno = [thresh_occ,thresh_vir]
# print(f'# lno thresh {thresh_pno}')

# if run_frg_list is None:
#     run_frg_list = range(nfrag)

# from jax import random
# seeds = random.randint(random.PRNGKey(options["seed"]),
#                     shape=(len(run_frg_list),), minval=0, maxval=100*nfrag)

# for ifrag in run_frg_list:
#     print(f'\n########### running fragment {ifrag+1} ##########')

#     fraglo = frag_lolist[ifrag]
#     orbfragloc = orbloc[:,fraglo] # the specific local active occ
#     # frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
#     THRESH_INTERNAL = 1e-10
#     frzfrag, orbfrag, can_orbfrag \
#         = lno_ccsd.make_lno(lno_cc, orbfragloc, THRESH_INTERNAL, thresh_pno)
    
#     mol = mf.mol
#     nocc = mol.nelectron // 2 
#     nao = mol.nao
#     actfrag = np.array([i for i in range(nao) if i not in frzfrag])
#     # frzocc = np.array([i for i in range(nocc) if i in frzfrag])
#     actocc = np.array([i for i in range(nocc) if i in actfrag])
#     actvir = np.array([i for i in range(nocc,nao) if i in actfrag])
#     nactocc = len(actocc)
#     nactocc = len(actocc)
#     nactvir = len(actvir)
#     prjlo = orbfragloc.T @ s1e @ orbfrag[:,actocc]

#     print(f'# active orbitals: {actfrag}')
#     print(f'# active occupied orbitals: {actocc}')
#     print(f'# active virtual orbitals: {actvir}')
#     print(f'# frozen orbitals: {frzfrag}')

#     if full_cisd:
#         # print('# This method is not size-extensive')
#         frz_mo_idx = np.where(np.array(frozen_mask) == False)[0]
#         act_mo_occ = np.array([i for i in range(nocc) if i not in frz_mo_idx])
#         act_mo_vir = np.array([i for i in range(nocc,nao) if i not in frz_mo_idx])
#         prj_no2mo = lno_ccsd.no2mo(mf.mo_coeff,s1e,orbfrag)
#         prj_oo_act = prj_no2mo[np.ix_(act_mo_occ,actocc)]
#         prj_vv_act = prj_no2mo[np.ix_(act_mo_vir,actvir)]
#         if isinstance(mfcc, CCSD):
#             print('# Use full CCSD wavefunction')
#             print('# Project CC amplitudes from MO to NO')
#             t1 = mfcc.t1
#             t2 = mfcc.t2
#             # project to active no
#             t1 = lib.einsum("ij,ia,ba->jb",prj_oo_act,t1,prj_vv_act.T)
#             t2 = lib.einsum("ik,jl,ijab,db,ca->klcd",
#                     prj_oo_act,prj_oo_act,t2,prj_vv_act.T,prj_vv_act.T)
#             ci1 = np.array(t1)
#             ci2 = t2 + lib.einsum("ia,jb->ijab",ci1,ci1)
#             ci2 = ci2.transpose(0, 2, 1, 3)
#         if isinstance(mfcc, CISD):
#             print('# Use full CISD wavefunction')
#             print('# Project CI coefficients from MO to NO')
#             v_ci = mfcc.ci
#             ci0,ci1,ci2 = mfcc.cisdvec_to_amplitudes(v_ci)
#             ci1 = ci1/ci0
#             ci2 = ci2/ci0
#             ci1 = lib.einsum("ij,ia,ba->jb",prj_oo_act,ci1,prj_vv_act.T)
#             ci2 = lib.einsum("ik,jl,ijab,db,ca->klcd",
#                     prj_oo_act,prj_oo_act,ci2,prj_vv_act.T,prj_vv_act.T)
#             ci2 = ci2.transpose(0, 2, 1, 3)
#         print('# Finished MO to NO projection')
#         ecorr_ccsd = '  None  '
#     else:
#         print('# Solving LNO-CCSD')
#         ecorr_ccsd,t1,t2 = lno_ccsd.cc_impurity_solve(
#                 mf,orbfrag,orbfragloc,frozen=frzfrag,eris=None
#                 )
#         ci1 = np.array(t1)
#         ci2 = t2 + lib.einsum("ia,jb->ijab",ci1,ci1)
#         ci2 = ci2.transpose(0, 2, 1, 3)
#         ecorr_ccsd = f'{ecorr_ccsd:.8f}'
#         print(f'# lno-ccsd fragment correlation energy: {ecorr_ccsd}')

#     nelec_act = nactocc*2
#     norb_act = nactocc+nactvir
    
#     print(f'# number of active electrons: {nelec_act}')
#     print(f'# number of active orbitals: {norb_act}')
#     print(f'# number of frozen orbitals: {len(frzfrag)}')

#     options["seed"] = seeds[ifrag]
#     lno_ccsd.prep_lno_amp_chol_file(
#         mf,orbfrag,options,
#         norb_act=norb_act,nelec_act=nelec_act,
#         prjlo=prjlo,norb_frozen=frzfrag,
#         ci1=ci1,ci2=ci2,use_df_vecs=use_df_vecs,
#         chol_cut=chol_cut,
#         )


lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228


macro= 1  f(x)= 24.514112851113  delta_f= 24.5141  |g|= 10.0677  2 KF 10 Hx
macro= 2  f(x)= 22.685522230903  delta_f= -1.82859  |g|= 4.39755  2 KF 10 Hx
macro= 3  f(x)= 22.434500027171  delta_f= -0.251022  |g|= 0.174092  2 KF 7 Hx
macro= 4  f(x)= 22.434499596429  delta_f= -4.30742e-07  |g|= 0.00393928  1 KF 2 Hx
macro= 5  f(x)= 22.434499596429  delta_f= 5.68434e-14  |g|= 1.09843e-07  1 KF 1 Hx
macro X = 5  f(x)= 22.434499596429  |g|= 1.09843e-07  10 intor 8 KF 30 Hx
LOs span exactly the occupied space.
Using single-LO fragment
# lno thresh [0, 0]

########### running fragment 1 ##########
Using true 4-index integrals
Using true 4-index integrals
# active orbitals: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
# active occupied orbitals: [0 1 2 3 4]
# active virtual orbitals: [ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39]
# frozen orbitals: []
# Use f

In [ ]:
# from functools import partial
# from jax import random
# #from mpi4py import MPI
# import numpy as np
# from jax import numpy as jnp
# from ad_afqmc import config, wavefunctions, stat_utils
# from ad_afqmc.lno_ccsd import lno_ccsd
# import time

# print = partial(print, flush=True)

# ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = (
#     lno_ccsd.prep_lnoccsd_afqmc())

# if options["use_gpu"]:
#     config.afqmc_config["use_gpu"] = True

# config.setup_jax()
# MPI = config.setup_comm()

# comm = MPI.COMM_WORLD
# rank = comm.Get_rank()  # Process rank
# size = comm.Get_size()  # Total number of processes

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 40
# nelec: (5, 5)
# nchol: 207
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 10
# seed: 450
# walker_type: rhf
# trial: rhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# n_batch: 1
# ene0: 0
#
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


In [ ]:
# ### initialize propagation
# seed = options["seed"]
# propagator = prop
# init_walkers = None
# ham_data = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
#                                 )._build_measurement_intermediates(ham_data, wave_data)
# ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
# ham_data = ham.build_propagation_intermediates(ham_data, propagator, trial, wave_data)

# prop_data = propagator.init_prop_data(trial, wave_data, ham_data, init_walkers)


In [ ]:
# t2_0 = np.zeros(t2.shape)
# eris = mycc.ao2mo(orbfrag)
# ccsd_corr = mycc.energy(t1,t2,eris)
# print(mf.e_tot+ccsd_corr)
# print(mycc.e_tot)

-5.541455769668331
-5.541455769668331


In [ ]:
# t2_0 = np.zeros(t2.shape)
# eris = mycc.ao2mo(orbfrag)
# ccsd_corr = mycc.energy(t1,t2_0,eris)
# print(mf.e_tot+ccsd_corr)
# t2_0 = np.zeros(t2.shape)
# eris = mycc.ao2mo(mycc.mo_coeff)
# ccsd_corr = mycc.energy(mycc.t1,t2_0,eris)
# print(mf.e_tot+ccsd_corr)

-5.370456322958685
-5.3704563229586855


In [ ]:
# print(mf.e_tot)

-5.370441409832113


In [ ]:
# u_occ = thouless_trans(t1)

In [ ]:
# e_phi \
# = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
#                     )._calc_energy_restricted(u_occ, ham_data, wave_data)
# print(e_phi)
# print(e_phi-(mf.e_tot+ccsd_corr))

-5.370456297319951
2.56387346908582e-08


In [ ]:
# use phi = exp(T1)psi_0 as the trial
# the scattering elements of <0|1> is not zero!!!

In [ ]:
# get the transformation of u_vir
# def thouless_trans(t1):
#     q, r = np.linalg.qr(t1)
#     u_ai = r.T
#     u_ji = q
#     u_occ = np.vstack((u_ji,u_ai))
#     u, _, _ = np.linalg.svd(u_occ)
#     return u

In [ ]:
# u = thouless_trans(t1)

In [ ]:
# nocc = np.count_nonzero(mf.mo_occ)
# # print(u[:,:nocc]-u_occ)
# e_phi \
#     = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
#                     )._calc_energy_restricted(u[:,:nocc], ham_data, wave_data)
# print(e_phi)
# print(e_phi-(mf.e_tot+ccsd_corr))

-5.370456297319951
2.56387346908582e-08


In [ ]:
# mfcc = mycc
# thresh = 0
# frozen = None
# run_frg_list = [0]
# use_df_vecs = False
# chol_cut = 1e-8
# lo_type = 'boys'
# no_type = 'ie'
# frag_lolist = '1o'

# from pyscf.cc.ccsd import CCSD
# from pyscf.ci.cisd import CISD
# if isinstance(mfcc, (CCSD, CISD)):
#     full_cisd = True
#     mf = mfcc._scf
# else:
#     full_cisd = False
#     mf = mfcc

# if isinstance(thresh, list):
#     thresh_occ, thresh_vir = thresh
# else:
#     thresh_occ = thresh*10
#     thresh_vir = thresh

# lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen)
# lno_cc.thresh_occ = thresh_occ
# lno_cc.thresh_vir = thresh_vir
# lno_cc.lo_type = lo_type
# lno_cc.no_type = no_type
# lno_cc.frag_lolist = frag_lolist
# lno_cc.force_outcore_ao2mo = True

# s1e = lno_cc._scf.get_ovlp()
# orbactocc = lno_cc.split_mo()[1] # non-localized active occ
# orbloc = lno_cc.get_lo(lo_type=lo_type) # localized active occ orbitals
# m = orbloc.T @ s1e @ orbactocc
# lospanerr = abs((m.T @ m) - np.eye(m.shape[1])).max()
# if lospanerr > 1e-10:
#     log.error('LOs do not fully span the occupied space! '
#                 'Max|<occ|LO><LO|occ>| = %e', lospanerr)
#     raise RuntimeError
# # check 2: Span(LO) == Span(occ)
# occspanerr = abs(m @ m.T - np.eye(m.shape[0])).max()
# if occspanerr < 1e-10:
#     log.info('LOs span exactly the occupied space.')
#     if no_type not in ['ir','ie']:
#         log.error('"no_type" must be "ir" or "ie".')
#         raise ValueError
# else:
#     log.info('LOs span occupied space plus some virtual space.')

# if frag_lolist == '1o':
#     log.info('Using single-LO fragment') # this is what we use, every active local occ labels a fragment
#     frag_lolist = [[i] for i in range(orbloc.shape[1])]
# else: print('Only support single LO fragment!')
# nfrag = len(frag_lolist)

# if not (no_type[0] in 'rei' and no_type[1] in 'rei'):
#     log.warn('Input no_type "%s" is invalid.', no_type)
#     raise ValueError

# frozen_mask = lno_cc.get_frozen_mask()
# thresh_pno = [thresh_occ,thresh_vir]
# print(f'# lno thresh {thresh_pno}')

# if run_frg_list is None:
#     run_frg_list = range(nfrag)

# from jax import random
# seeds = random.randint(random.PRNGKey(options["seed"]),
#                     shape=(len(run_frg_list),), minval=0, maxval=100*nfrag)

# for ifrag in run_frg_list:
#     print(f'\n########### running fragment {ifrag+1} ##########')

#     fraglo = frag_lolist[ifrag]
#     orbfragloc = orbloc[:,fraglo] # the specific local active occ
#     # frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
#     THRESH_INTERNAL = 1e-10
#     frzfrag, orbfrag, can_orbfrag \
#         = lno_ccsd.make_lno(lno_cc, orbfragloc, THRESH_INTERNAL, thresh_pno)
    
#     mol = mf.mol
#     nocc = mol.nelectron // 2 
#     nao = mol.nao
#     actfrag = np.array([i for i in range(nao) if i not in frzfrag])
#     # frzocc = np.array([i for i in range(nocc) if i in frzfrag])
#     actocc = np.array([i for i in range(nocc) if i in actfrag])
#     actvir = np.array([i for i in range(nocc,nao) if i in actfrag])
#     nactocc = len(actocc)
#     nactocc = len(actocc)
#     nactvir = len(actvir)
#     prjlo = orbfragloc.T @ s1e @ orbfrag[:,actocc]

#     print(f'# active orbitals: {actfrag}')
#     print(f'# active occupied orbitals: {actocc}')
#     print(f'# active virtual orbitals: {actvir}')
#     print(f'# frozen orbitals: {frzfrag}')

#     if full_cisd:
#         # print('# This method is not size-extensive')
#         frz_mo_idx = np.where(np.array(frozen_mask) == False)[0]
#         act_mo_occ = np.array([i for i in range(nocc) if i not in frz_mo_idx])
#         act_mo_vir = np.array([i for i in range(nocc,nao) if i not in frz_mo_idx])
#         prj_no2mo = lno_ccsd.no2mo(mf.mo_coeff,s1e,orbfrag)
#         prj_oo_act = prj_no2mo[np.ix_(act_mo_occ,actocc)]
#         prj_vv_act = prj_no2mo[np.ix_(act_mo_vir,actvir)]
#         if isinstance(mfcc, CCSD):
#             print('# Use full CCSD wavefunction')
#             print('# Project CC amplitudes from MO to NO')
#             t1 = mfcc.t1
#             t2 = mfcc.t2
#             # project to active no
#             t1 = lib.einsum("ij,ia,ba->jb",prj_oo_act,t1,prj_vv_act.T)
#             t2 = lib.einsum("ik,jl,ijab,db,ca->klcd",
#                     prj_oo_act,prj_oo_act,t2,prj_vv_act.T,prj_vv_act.T)
#             ci1 = np.array(t1)
#             ci2 = t2 + lib.einsum("ia,jb->ijab",ci1,ci1)
#             ci2 = ci2.transpose(0, 2, 1, 3)
#         if isinstance(mfcc, CISD):
#             print('# Use full CISD wavefunction')
#             print('# Project CI coefficients from MO to NO')
#             v_ci = mfcc.ci
#             ci0,ci1,ci2 = mfcc.cisdvec_to_amplitudes(v_ci)
#             ci1 = ci1/ci0
#             ci2 = ci2/ci0
#             ci1 = lib.einsum("ij,ia,ba->jb",prj_oo_act,ci1,prj_vv_act.T)
#             ci2 = lib.einsum("ik,jl,ijab,db,ca->klcd",
#                     prj_oo_act,prj_oo_act,ci2,prj_vv_act.T,prj_vv_act.T)
#             ci2 = ci2.transpose(0, 2, 1, 3)
#         print('# Finished MO to NO projection')
#         ecorr_ccsd = '  None  '
#     else:
#         print('# Solving LNO-CCSD')
#         ecorr_ccsd,t1,t2 = lno_ccsd.cc_impurity_solve(
#                 mf,orbfrag,orbfragloc,frozen=frzfrag,eris=None
#                 )
#         ci1 = np.array(t1)
#         ci2 = t2 + lib.einsum("ia,jb->ijab",ci1,ci1)
#         ci2 = ci2.transpose(0, 2, 1, 3)
#         ecorr_ccsd = f'{ecorr_ccsd:.8f}'
#         print(f'# lno-ccsd fragment correlation energy: {ecorr_ccsd}')

#     nelec_act = nactocc*2
#     norb_act = nactocc+nactvir
    
#     print(f'# number of active electrons: {nelec_act}')
#     print(f'# number of active orbitals: {norb_act}')
#     print(f'# number of frozen orbitals: {len(frzfrag)}')

#     options["seed"] = seeds[ifrag]
#     lno_ccsd.prep_lno_amp_chol_file(
#         mf,orbfrag,options,
#         norb_act=norb_act,nelec_act=nelec_act,
#         prjlo=prjlo,norb_frozen=frzfrag,
#         ci1=ci1,ci2=ci2,use_df_vecs=use_df_vecs,
#         chol_cut=chol_cut,
#         )